In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as sp
from scipy.stats import norm
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import pearsonr
import mne

def sine_func(fs, freq, amp, time):
    samples = fs*time
    time = np.arange(samples)
    voltage = amp * np.sin(2 * np.pi * freq * time / fs)
    
    return time, voltage

## testing old COGS280 data

In [ ]:
# Whole time series -- 30390 time points
raw_EO = mne.io.read_raw_eeglab(r'C:\Users\Jake\testing-learning2024\Data\MichaelEOxdf.set', verbose = False)

channels = getattr(raw_EO, "ch_names")
n_channels = len(channels)
tau = 10
time_points = len(raw_EO)
print(time_points)
Forw = np.zeros((n_channels, n_channels))
Rev = np.zeros((n_channels, n_channels))

forw_data = raw_EO.get_data()
rev_data = np.flip(forw_data, 1)

for i in range(n_channels):
    for j in range(n_channels):
      x = pearsonr(forw_data[i,0:time_points-tau], forw_data[j,tau:time_points])
      y = pearsonr(rev_data[i,0:time_points-tau], rev_data[j,tau:time_points])
      Forw[i,j] = x[0]
      Rev[i,j] = y[0]
Forw = -1/2 * np.log(1 - (Forw**2))
Rev = -1/2 * np.log(1 - (Rev**2))
ref_matrix = (Forw - Rev)**2
irrev = np.mean(ref_matrix)
stationarity = np.std(ref_matrix)
print("EO irrev: " + str(irrev))


## Autocorrelation function testing

In [ ]:
data = mne.io.read_raw_eeglab(r'C:\Users\Jake\testing-learning2024\Data\MichaelEOxdf.set', verbose = False)
data = data.get_data()

def autocorrelation(x, max_lag):
    """ Compute autocorrelation using NumPy for a given max_lag """
    autocorr = np.correlate(x - np.mean(x), x - np.mean(x), mode='full')
    autocorr = autocorr[autocorr.size // 2:]  # Take the second half
    autocorr /= autocorr[0]  # Normalize
    return autocorr[:max_lag+1]

max_lag = 25  # Adjust based on your data and needs
autocorr_values = autocorrelation(data[0,:], max_lag)

plt.figure(figsize=(10, 5))
plt.plot(autocorr_values, linestyle='-', marker='o')
plt.title('Autocorrelation of EEG Data')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')
plt.grid(True)
plt.show()